<a href="https://colab.research.google.com/github/reo113/GuideHouse_Project/blob/elena-branch/GuideHouse_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q xlrd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
!git clone https://github.com/reo113/GuideHouse_Project

fatal: destination path 'GuideHouse_Project' already exists and is not an empty directory.


In [59]:
#%load GuideHouse_Project/guidehouse.ipynb

In [60]:
def getDate(filename):
    match_str = re.search(r'\d{4}-\d{2}-\d{2}',filename)
    try:
        return datetime.datetime.strptime(match_str.group(), '%Y-%m-%d').date()
    except ValueError:
        return None


In [61]:
def cleanColumn(name):
    name = name.upper()
    clean_name = re.sub(r'[^a-zA-Z0-9_]', '', name)
    clean_name = clean_name.replace('\n', ' ')
    if clean_name == 'SHELTERNFI':
        clean_name = 'SHELTER'
    if clean_name == 'PROTECTIONCP':
        clean_name = 'PC_CP'
    if clean_name == 'PROTECTIONGBV':
        clean_name = 'PC_GBV'
    if clean_name == 'PROTECTIONMA':
        clean_name = 'PC_MA'
    if clean_name == 'TOTALREACHED':
        clean_name = 'PEOPLEREACHED'
    return clean_name

In [62]:
#using csv and xlsx
#changed csv_dir
csv_dir = '/content/GuideHouse_Project/ukraine_data_excel'


In [63]:
#using xlsx
sheet1_df = []
sheet2_df = []
sheet3_df = []

In [64]:


count=0
for filename in os.listdir(csv_dir):

  print(filename)

  if filename.endswith('.xlsx'):
    file_path = os.path.join(csv_dir,filename)
    excel_df = pd.ExcelFile(file_path)
    files_date = getDate(filename)
    for sheet in excel_df.sheet_names:
      df = pd.read_excel(excel_df,sheet_name=sheet)
      df.columns = [cleanColumn(col) for col in df.columns]
      df['DATE'] = files_date
      if sheet == 'Num_of_Orgs_by_Oblast':
        sheet1_df.append(df)
      elif sheet == 'People_Reached_by_Oblast':
        sheet2_df.append(df)
      elif sheet == 'UDE_Inputs':
        sheet3_df.append(df)



Data_ Round 16 - Ukraine 5W - 2022-06-23.xlsx
Data_ Round 11 - Ukraine 5W - 2022-05-19.xlsx
Data_ Round 8 - Ukraine 5W - 2022-04-28.xlsx
Data_ Round 21 - Ukraine 5W - 2022-07-28.xlsx
Data_ Round 15 - Ukraine 5W - 2022-06-16.xlsx
Data_ Round 18 - Ukraine 5W - 2022-07-07.xlsx
Data_ Round 36 - Ukraine 5W - 2022-12-22.xlsx
Data_ Round 24 - Ukraine 5W - 2022-08-18.xlsx
Data_ Round 14 - Ukraine 5W - 2022-06-09.xlsx
Data_ Round 28 - Ukraine 5W - 2022-09-22.xlsx
Data_ Round 37 - Ukraine 5W - 2023-01-05.xlsx
Data_ Round 32 - Ukraine 5W - 2022-10-27.xlsx
Data_ Round 12 - Ukraine 5W - 2022-05-26.xlsx
Data_ Round 17 - Ukraine 5W - 2022-06-30.xlsx
Data_ Round 20 - Ukraine 5W - 2022-07-21.xlsx
Data_ Round 33 - Ukraine 5W - 2022-11-10.xlsx
Data_ Round 29 - Ukraine 5W - 2022-09-29.xlsx
Data_ Round 19 - Ukraine 5W - 2022-07-14.xlsx
Data_ Round 35 - Ukraine 5W - 2022-12-08.xlsx
Data_ Round 31 - Ukraine 5W - 2022-10-13.xlsx
Data_ Round 13 - Ukraine 5W - 2022-06-02.xlsx
Data_ Round 26 - Ukraine 5W - 2022-

In [65]:
combined_sheet1 = pd.concat(sheet1_df, keys=[str(df['DATE'].iloc[0]) for df in sheet1_df])
res_sheet1 = combined_sheet1.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()
merged_sheet1 = pd.merge(res_sheet1, combined_sheet1[['ADMIN1_ID', 'DATE', 'OBLAST']], on=['ADMIN1_ID', 'DATE'], how='left')


<ipython-input-65-f126ab6578e9>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_sheet1 = combined_sheet1.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()


In [69]:
combined_sheet2 = pd.concat(sheet2_df, keys=[str(df['DATE'].iloc[0]) for df in sheet2_df])
res_sheet2 = combined_sheet2.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()
merged_sheet2 = pd.merge(res_sheet2, combined_sheet2[['ADMIN1_ID', 'DATE', 'OBLAST']], on=['ADMIN1_ID', 'DATE'], how='left')


<ipython-input-69-86453b5296f8>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_sheet2 = combined_sheet2.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()


In [73]:
combined_sheet3 = pd.concat(sheet3_df, keys=[str(df['DATE'].iloc[0]) for df in sheet3_df])
res_sheet3 = combined_sheet3.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()
merged_sheet3 = pd.merge(res_sheet3, combined_sheet3[['ADMIN1_ID', 'DATE', 'OBLAST']], on=['ADMIN1_ID', 'DATE'], how='left')


<ipython-input-73-b2b5d3c940f8>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_sheet3 = combined_sheet3.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()


In [74]:
merged_sheet1.shape

(750, 20)

In [75]:
merged_sheet3.shape


(750, 5)

In [76]:
merged_sheet1.iloc[26]


ADMIN1_ID                UA05
DATE               2022-11-25
CCCM                      0.0
CCS                       0.0
ETC                       0.0
EDUCATION                 0.0
FSL                       0.0
HEALTH                    0.0
LOGISTICS                 0.0
MPC                       0.0
NUTRITION                 0.0
SHELTER                   0.0
PROTECTIONTOTAL           0.0
PC_CP                     0.0
PC_GBV                    0.0
PC_MA                     0.0
WASH                      0.0
TOTAL                     0.0
PC_PC                     0.0
OBLAST              Vinnytska
Name: 26, dtype: object

In [78]:
#Num_of_Orgs_by_Oblast
merged_sheet1.iloc[20:35]


,ADMIN1_ID,DATE,CCCM,CCS,ETC,EDUCATION,FSL,HEALTH,LOGISTICS,MPC,NUTRITION,SHELTER,PROTECTIONTOTAL,PC_CP,PC_GBV,PC_MA,WASH,TOTAL,PC_PC,OBLAST
20,UA05,2022-09-22,2.0,1.0,0.0,4.0,20.0,13.0,0.0,6.0,0.0,7.0,0.0,4.0,3.0,3.0,9.0,58.0,0.0,Vinnytska
21,UA05,2022-09-29,6.0,2.0,0.0,20.0,68.0,48.0,0.0,34.0,0.0,46.0,0.0,36.0,14.0,8.0,30.0,214.0,0.0,Vinnytska
22,UA05,2022-09-29,6.0,2.0,0.0,20.0,68.0,48.0,0.0,34.0,0.0,46.0,0.0,36.0,14.0,8.0,30.0,214.0,0.0,Vinnytska
23,UA05,2022-10-13,3.0,1.0,0.0,11.0,35.0,24.0,0.0,17.0,0.0,23.0,0.0,25.0,16.0,5.0,13.0,123.0,0.0,Vinnytska
24,UA05,2022-10-27,3.0,1.0,0.0,13.0,36.0,24.0,0.0,17.0,0.0,23.0,0.0,25.0,10.0,5.0,13.0,119.0,0.0,Vinnytska
25,UA05,2022-11-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vinnytska
26,UA05,2022-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vinnytska
27,UA05,2022-12-08,3.0,1.0,0.0,13.0,36.0,27.0,0.0,18.0,0.0,24.0,0.0,26.0,11.0,7.0,13.0,125.0,0.0,Vinnytska
28,UA05,2022-12-22,2.0,1.0,0.0,13.0,36.0,33.0,0.0,18.0,0.0,24.0,0.0,29.0,11.0,8.0,13.0,130.0,0.0,Vinnytska
29,UA05,2023-01-05,3.0,1.0,1.0,17.0,39.0,33.0,0.0,19.0,0.0,25.0,0.0,33.0,10.0,8.0,14.0,138.0,0.0,Vinnytska


In [79]:
merged_sheet3.iloc[20:35]


,ADMIN1_ID,DATE,PEOPLEREACHED,NUMBEROFORGANISATIONS,OBLAST
20,UA05,2022-09-22,395394,58,Vinnytska
21,UA05,2022-09-29,790788,214,Vinnytska
22,UA05,2022-09-29,790788,214,Vinnytska
23,UA05,2022-10-13,395394,123,Vinnytska
24,UA05,2022-10-27,395394,119,Vinnytska
25,UA05,2022-11-10,449220,125,Vinnytska
26,UA05,2022-11-25,449220,125,Vinnytska
27,UA05,2022-12-08,449220,125,Vinnytska
28,UA05,2022-12-22,449220,130,Vinnytska
29,UA05,2023-01-05,449220,138,Vinnytska


In [80]:
merged_sheet1['OBLAST'].value_counts()


Vinnytska           30
Odeska              30
Chernihivska        30
Chernivetska        30
Cherkaska           30
Khmelnytska         30
Khersonska          30
Kharkivska          30
Ternopilska         30
Sumska              30
Rivnenska           30
Poltavska           30
Mykolaivska         30
Volynska            30
Lvivska             30
Luhanska            30
Kirovohradska       30
Kyivska             30
Ivano-Frankivska    30
Zaporizka           30
Zakarpatska         30
Zhytomyrska         30
Donetska            30
Dnipropetrovska     30
Kyiv                30
Name: OBLAST, dtype: int64

In [81]:
sum(merged_sheet3['PEOPLEREACHED'].value_counts())


750

In [82]:
# replace zero values in the empty tuple to NAN for future linear interpolation
#NOTE : THIS IS HARD CODED NOT OPTIMAL
merged_sheet1.iloc[25] = merged_sheet1.iloc[25].replace(0,np.nan)
merged_sheet1.iloc[26] = merged_sheet1.iloc[26].replace(0,np.nan)

merged_sheet3.iloc[25] = merged_sheet3.iloc[25].replace(merged_sheet3.iloc[25]['PEOPLEREACHED'],np.nan)
merged_sheet3.iloc[26] = merged_sheet3.iloc[26].replace(merged_sheet3.iloc[26]['PEOPLEREACHED'],np.nan)


In [83]:
merged_sheet1['PEOPLE_REACHED'] = merged_sheet3['PEOPLEREACHED']


In [84]:
#check for null values
merged_sheet1.isna().any().any()

True

In [85]:
# check for null values in specific row
merged_sheet1.iloc[26].isna().any()


True

In [86]:
#filled null values using linear interpolation
interpolated_df = merged_sheet1.interpolate(method='linear')


In [87]:
interpolated_df.isna().any().any()


False

In [88]:
# rounded all int and float dtypes since interpolation caused floats
interpolated_df = interpolated_df.applymap(lambda x: round(x, 1) if isinstance(x, (int, float)) else x)


In [89]:
interpolated_df.iloc[20:35]


,ADMIN1_ID,DATE,CCCM,CCS,ETC,EDUCATION,FSL,HEALTH,LOGISTICS,MPC,...,SHELTER,PROTECTIONTOTAL,PC_CP,PC_GBV,PC_MA,WASH,TOTAL,PC_PC,OBLAST,PEOPLE_REACHED
20,UA05,2022-09-22,2.0,1.0,0.0,4.0,20.0,13.0,0.0,6.0,...,7.0,0.0,4.0,3.0,3.0,9.0,58.0,0.0,Vinnytska,395394.0
21,UA05,2022-09-29,6.0,2.0,0.0,20.0,68.0,48.0,0.0,34.0,...,46.0,0.0,36.0,14.0,8.0,30.0,214.0,0.0,Vinnytska,790788.0
22,UA05,2022-09-29,6.0,2.0,0.0,20.0,68.0,48.0,0.0,34.0,...,46.0,0.0,36.0,14.0,8.0,30.0,214.0,0.0,Vinnytska,790788.0
23,UA05,2022-10-13,3.0,1.0,0.0,11.0,35.0,24.0,0.0,17.0,...,23.0,0.0,25.0,16.0,5.0,13.0,123.0,0.0,Vinnytska,395394.0
24,UA05,2022-10-27,3.0,1.0,0.0,13.0,36.0,24.0,0.0,17.0,...,23.0,0.0,25.0,10.0,5.0,13.0,119.0,0.0,Vinnytska,395394.0
25,UA05,2022-11-10,3.0,1.0,0.0,13.0,36.0,25.0,0.0,17.3,...,23.3,0.0,25.3,10.3,5.7,13.0,121.0,0.0,Vinnytska,413336.0
26,UA05,2022-11-25,3.0,1.0,0.0,13.0,36.0,26.0,0.0,17.7,...,23.7,0.0,25.7,10.7,6.3,13.0,123.0,0.0,Vinnytska,431278.0
27,UA05,2022-12-08,3.0,1.0,0.0,13.0,36.0,27.0,0.0,18.0,...,24.0,0.0,26.0,11.0,7.0,13.0,125.0,0.0,Vinnytska,449220.0
28,UA05,2022-12-22,2.0,1.0,0.0,13.0,36.0,33.0,0.0,18.0,...,24.0,0.0,29.0,11.0,8.0,13.0,130.0,0.0,Vinnytska,449220.0
29,UA05,2023-01-05,3.0,1.0,1.0,17.0,39.0,33.0,0.0,19.0,...,25.0,0.0,33.0,10.0,8.0,14.0,138.0,0.0,Vinnytska,449220.0


In [90]:
interpolated_df.sort_values(by=['DATE'])


,ADMIN1_ID,DATE,CCCM,CCS,ETC,EDUCATION,FSL,HEALTH,LOGISTICS,MPC,...,SHELTER,PROTECTIONTOTAL,PC_CP,PC_GBV,PC_MA,WASH,TOTAL,PC_PC,OBLAST,PEOPLE_REACHED
0,UA05,2022-04-28,2.0,1.0,1.0,5.0,17.0,16.0,0.0,10.0,...,6.0,16.0,2.0,2.0,2.0,7.0,42.0,11.0,Vinnytska,91200.0
690,UA74,2022-04-28,1.0,0.0,0.0,1.0,17.0,12.0,0.0,6.0,...,2.0,9.0,1.0,0.0,1.0,10.0,36.0,8.0,Chernihivska,171600.0
60,UA12,2022-04-28,1.0,1.0,1.0,3.0,31.0,19.0,0.0,12.0,...,10.0,21.0,5.0,4.0,2.0,15.0,58.0,14.0,Dnipropetrovska,231800.0
660,UA73,2022-04-28,2.0,1.0,0.0,3.0,22.0,9.0,0.0,10.0,...,6.0,17.0,5.0,2.0,2.0,11.0,50.0,12.0,Chernivetska,31200.0
90,UA14,2022-04-28,0.0,1.0,0.0,2.0,48.0,26.0,0.0,12.0,...,9.0,29.0,7.0,4.0,1.0,18.0,75.0,21.0,Donetska,122100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,UA46,2023-01-05,4.0,1.0,1.0,23.0,62.0,52.0,1.0,26.0,...,27.0,0.0,41.0,20.0,8.0,23.0,194.0,0.0,Lvivska,957877.0
569,UA63,2023-01-05,3.0,0.0,1.0,13.0,111.0,49.0,0.0,24.0,...,33.0,0.0,26.0,7.0,10.0,25.0,175.0,0.0,Kharkivska,2070173.0
719,UA74,2023-01-05,0.0,0.0,0.0,18.0,56.0,29.0,0.0,20.0,...,33.0,0.0,16.0,7.0,9.0,17.0,149.0,0.0,Chernihivska,477499.0
89,UA12,2023-01-05,3.0,1.0,1.0,19.0,113.0,52.0,1.0,33.0,...,51.0,0.0,40.0,12.0,9.0,32.0,244.0,0.0,Dnipropetrovska,847604.0


In [91]:
interpolated_df.describe()


,CCCM,CCS,ETC,EDUCATION,FSL,HEALTH,LOGISTICS,MPC,NUTRITION,SHELTER,PROTECTIONTOTAL,PC_CP,PC_GBV,PC_MA,WASH,TOTAL,PC_PC,PEOPLE_REACHED
count,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,7.500000e+02
mean,0.908000,0.381333,0.192000,6.189333,34.133333,19.610667,0.233333,13.705333,1.602667,12.088000,5.958667,11.078667,5.553333,4.140000,12.689333,77.418667,1.804000,4.709115e+05
std,1.287016,0.538180,0.420363,5.379970,27.577556,14.599793,0.465307,9.082728,1.544876,9.815506,9.210034,9.966830,4.454329,2.705994,8.739412,53.084492,4.586243,4.379627e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.380000e+04
25%,0.000000,0.000000,0.000000,2.000000,18.000000,10.000000,0.000000,9.000000,0.000000,6.000000,0.000000,4.000000,2.000000,2.000000,7.000000,42.000000,0.000000,2.179775e+05
50%,0.000000,0.000000,0.000000,5.000000,29.000000,16.000000,0.000000,12.000000,1.000000,10.000000,0.000000,8.000000,5.000000,4.000000,12.000000,66.000000,0.000000,3.837110e+05
75%,1.000000,1.000000,0.000000,9.000000,44.000000,25.000000,0.000000,18.000000,2.000000,16.000000,13.000000,16.000000,8.000000,5.525000,16.000000,101.000000,0.000000,5.839760e+05
max,8.000000,2.000000,2.000000,32.000000,202.000000,100.000000,2.000000,56.000000,10.000000,80.000000,42.000000,52.000000,30.000000,16.000000,58.000000,364.000000,28.000000,4.140346e+06


In [93]:
# create label encoder instance
label_encoder = LabelEncoder()
#fit label encoder to transform Oblast column
encoded_oblast = label_encoder.fit_transform(interpolated_df['OBLAST'])

#create copy
df_encoded = interpolated_df.copy()

#Add column and assign values
df_encoded['OBLAST_ENCODED'] = encoded_oblast


In [94]:
# create label encoder instance
label_encoder = LabelEncoder()

#fit label encoder to transform Date column
encoded_oblast = label_encoder.fit_transform(df_encoded['DATE'])

#create copy
df_encode = df_encoded.copy()

#Add column and assign values
df_encode['DATE_ENCODED'] = encoded_oblast


In [95]:
df_encode.sort_values(by=['DATE'])


,ADMIN1_ID,DATE,CCCM,CCS,ETC,EDUCATION,FSL,HEALTH,LOGISTICS,MPC,...,PC_CP,PC_GBV,PC_MA,WASH,TOTAL,PC_PC,OBLAST,PEOPLE_REACHED,OBLAST_ENCODED,DATE_ENCODED
0,UA05,2022-04-28,2.0,1.0,1.0,5.0,17.0,16.0,0.0,10.0,...,2.0,2.0,2.0,7.0,42.0,11.0,Vinnytska,91200.0,20,0
690,UA74,2022-04-28,1.0,0.0,0.0,1.0,17.0,12.0,0.0,6.0,...,1.0,0.0,1.0,10.0,36.0,8.0,Chernihivska,171600.0,1,0
60,UA12,2022-04-28,1.0,1.0,1.0,3.0,31.0,19.0,0.0,12.0,...,5.0,4.0,2.0,15.0,58.0,14.0,Dnipropetrovska,231800.0,3,0
660,UA73,2022-04-28,2.0,1.0,0.0,3.0,22.0,9.0,0.0,10.0,...,5.0,2.0,2.0,11.0,50.0,12.0,Chernivetska,31200.0,2,0
90,UA14,2022-04-28,0.0,1.0,0.0,2.0,48.0,26.0,0.0,12.0,...,7.0,4.0,1.0,18.0,75.0,21.0,Donetska,122100.0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,UA46,2023-01-05,4.0,1.0,1.0,23.0,62.0,52.0,1.0,26.0,...,41.0,20.0,8.0,23.0,194.0,0.0,Lvivska,957877.0,13,28
569,UA63,2023-01-05,3.0,0.0,1.0,13.0,111.0,49.0,0.0,24.0,...,26.0,7.0,10.0,25.0,175.0,0.0,Kharkivska,2070173.0,6,28
719,UA74,2023-01-05,0.0,0.0,0.0,18.0,56.0,29.0,0.0,20.0,...,16.0,7.0,9.0,17.0,149.0,0.0,Chernihivska,477499.0,1,28
89,UA12,2023-01-05,3.0,1.0,1.0,19.0,113.0,52.0,1.0,33.0,...,40.0,12.0,9.0,32.0,244.0,0.0,Dnipropetrovska,847604.0,3,28


In [96]:
df_encoded.columns

Index(['ADMIN1_ID', 'DATE', 'CCCM', 'CCS', 'ETC', 'EDUCATION', 'FSL', 'HEALTH',
       'LOGISTICS', 'MPC', 'NUTRITION', 'SHELTER', 'PROTECTIONTOTAL', 'PC_CP',
       'PC_GBV', 'PC_MA', 'WASH', 'TOTAL', 'PC_PC', 'OBLAST', 'PEOPLE_REACHED',
       'OBLAST_ENCODED'],
      dtype='object')

In [97]:
merged_sheet1.shape


(750, 21)

In [98]:
#People_Reached_by_Oblast
merged_sheet2.head(10)

,ADMIN1_ID,DATE,FSL,CP,MINEACTION,PROTECTIONTOTAL,HEALTH,MPC,SHELTER,WASH,PEOPLEREACHED,GENERALPROTECTION,OBLAST
0,UA05,2022-04-28,7900.0,100.0,500.0,11900.0,4600.0,91200.0,15800.0,2300.0,91200.0,11200.0,Vinnytska
1,UA05,2022-05-06,14300.0,100.0,500.0,14300.0,4600.0,113000.0,16200.0,2300.0,113000.0,9600.0,Vinnytska
2,UA05,2022-05-12,15500.0,200.0,500.0,16300.0,9900.0,138800.0,17500.0,3700.0,138800.0,11400.0,Vinnytska
3,UA05,2022-05-19,17500.0,500.0,500.0,20300.0,23200.0,140600.0,17500.0,3700.0,140600.0,0.0,Vinnytska
4,UA05,2022-05-26,17600.0,4400.0,700.0,25600.0,23300.0,216600.0,21700.0,3800.0,216600.0,0.0,Vinnytska
5,UA05,2022-06-02,19000.0,21500.0,700.0,48000.0,23300.0,236000.0,33300.0,3800.0,236000.0,0.0,Vinnytska
6,UA05,2022-06-09,19000.0,21500.0,700.0,21000.0,23300.0,240100.0,40500.0,23800.0,240100.0,0.0,Vinnytska
7,UA05,2022-07-07,39200.0,46500.0,11800.0,94900.0,27500.0,214600.0,59400.0,42700.0,214600.0,0.0,Vinnytska
8,UA05,2022-07-14,39200.0,35700.0,11800.0,75700.0,27500.0,216000.0,60700.0,42700.0,216000.0,0.0,Vinnytska
9,UA05,2022-07-21,39200.0,35700.0,99800.0,174700.0,39000.0,217200.0,62600.0,42700.0,217200.0,0.0,Vinnytska


In [99]:
#UDE_Inputs
merged_sheet3.head()

,ADMIN1_ID,DATE,PEOPLEREACHED,NUMBEROFORGANISATIONS,OBLAST
0,UA05,2022-04-28,91200.0,42,Vinnytska
1,UA05,2022-05-06,113000.0,45,Vinnytska
2,UA05,2022-05-12,138800.0,49,Vinnytska
3,UA05,2022-05-19,140600.0,50,Vinnytska
4,UA05,2022-05-26,216600.0,48,Vinnytska


In [100]:
# copy_df = pd.DataFrame(np.nan, index=merged_sheet1.index,columns=merged_sheet1.columns)
# copy_df['DATE'] = merged_sheet1['DATE']
# copy_df['OBLAST'] = merged_sheet1['OBLAST']
# copy_df.head()

In [101]:
# # Create an Excel writer object
# excel_writer1 = pd.ExcelWriter('Data_ Round 33 - Ukraine 5W - 2022-11-10.xlsx', engine='xlsxwriter')
# excel_writer2 = pd.ExcelWriter('Data_ Round 34 - Ukraine 5W - 2022-11-25.xlsx', engine='xlsxwriter')
# # Write each DataFrame to a separate sheet in the Excel file
# copy_df.to_excel(excel_writer1, sheet_name='Num_of_Orgs_by_Oblast', index=False)
# copy_df.to_excel(excel_writer2, sheet_name='Num_of_Orgs_by_Oblast', index=False)


In [102]:
len(merged_sheet1.columns)
merged_sheet1.columns

Index(['ADMIN1_ID', 'DATE', 'CCCM', 'CCS', 'ETC', 'EDUCATION', 'FSL', 'HEALTH',
       'LOGISTICS', 'MPC', 'NUTRITION', 'SHELTER', 'PROTECTIONTOTAL', 'PC_CP',
       'PC_GBV', 'PC_MA', 'WASH', 'TOTAL', 'PC_PC', 'OBLAST',
       'PEOPLE_REACHED'],
      dtype='object')

In [103]:
merged_sheet1.shape


(750, 21)

In [104]:
merged_sheet3.shape

(750, 5)

In [105]:
merged_sheet1.DATE.value_counts()


2022-09-29    50
2022-04-28    25
2022-08-11    25
2022-12-22    25
2022-12-08    25
2022-11-25    25
2022-11-10    25
2022-10-27    25
2022-10-13    25
2022-09-22    25
2022-09-15    25
2022-09-08    25
2022-08-25    25
2022-08-18    25
2022-08-04    25
2022-05-06    25
2022-07-28    25
2022-07-21    25
2022-07-14    25
2022-07-07    25
2022-06-30    25
2022-06-23    25
2022-06-16    25
2022-06-09    25
2022-06-02    25
2022-05-26    25
2022-05-19    25
2022-05-12    25
2023-01-05    25
Name: DATE, dtype: int64

In [106]:
'''
personally working on PC_CP, PC_GBV, PC_MA
linear regression model on the labels

- PC_CP, PC_GBV, PC_MA
- using interpolated_df
'''

'\npersonally working on PC_CP, PC_GBV, PC_MA\nlinear regression model on the labels\n\n- PC_CP, PC_GBV, PC_MA\n- using interpolated_df\n'

In [107]:
!git commit -m 'fixed errors in vscode by transfering code to colab'  # commit in Colab
!git push origin master          # push to github

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
